In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Nov 13 21:43:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.6 MB/s 
     |████████████████████████████████| 7.6 MB 56.7 MB/s 
     |████████████████████████████████| 163 kB 56.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 23.3 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 115 kB 58.6 MB/s 
     |████████████████████████████████| 212 kB 8.6 MB/s 
     |████████████████████████████████| 127 kB 71.4 MB/s 
     |████████████████████████████████| 115 kB 78.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uni

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

In [ ]:
# raw_datasets = load_dataset("wmt16", "de-en")
raw_datasets = load_dataset("wmt16", "de-en")
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})


In [ ]:
raw_dataset_train = load_dataset("wmt16", "de-en", split="train[:20%]")
raw_dataset_test = load_dataset("wmt16", "de-en", split="test[:20%]")
raw_dataset_validation = load_dataset("wmt16", "de-en", split="validation[:20%]")

In [ ]:
print(raw_dataset_train)
print(raw_dataset_test)
print(raw_dataset_validation)

Dataset({
    features: ['translation'],
    num_rows: 909777
})
Dataset({
    features: ['translation'],
    num_rows: 600
})
Dataset({
    features: ['translation'],
    num_rows: 434
})


In [ ]:
from datasets import DatasetDict 
raw_datasets = DatasetDict({'train': raw_dataset_train, 'test': raw_dataset_test, 'validation': raw_dataset_validation})

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 909777
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 434
    })
})


In [ ]:
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi", "hello there !"],["foo bar foobar", "foo bar foobar"]]
results = metric.compute(predictions=predictions, references=references)
print(round(results["score"], 1))

100.0


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
prefix = "translate English to German: "

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk('./tokenizer')

FileNotFoundError: ignored

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/910 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets.save_to_disk('./tokenizer')

In [ ]:
print(tokenized_datasets['train'][0])

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}, 'input_ids': [13959, 1566, 12, 2968, 10, 419, 4078, 102, 1575, 13, 8, 2363, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [15158, 24860, 74, 11216, 425, 7, 4267, 32, 221, 1, 0, 0, 0, 0, 

In [ ]:
print(tokenizer.decode(tokenized_datasets['train'][0]['input_ids'], skip_special_tokens=True))

translate English to German: Resumption of the session


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small").cuda()

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained("tgummadi/t5-11785",trust_remote_code=True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch import nn
from torch.distributions import Categorical
import sys, torch

class CustomTrainer(Seq2SeqTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bleu = []
        self.distributions = []
        self.counter = 0 

    def compute_loss(self, model, inputs, return_outputs=False):
        log_probs = []
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        attn_mask = torch.where(labels != -100, inputs['attention_mask'], 0)
        masked_labels = labels*attn_mask
        # decoded_labels = []
        # for i in range(len(masked_labels)):
        #   decoded_labels.append([tokenizer.decode(masked_labels[i], skip_special_tokens=True)])
        decoded_labels = tokenizer.batch_decode(masked_labels,skip_special_tokens=True)
        decoded_labels = [[i] for i in decoded_labels]

        attn_mask = torch.unsqueeze(attn_mask, 2)
        masked_logits = logits*attn_mask
        decoded_logits = []
        log_probs_batch = []
        # for i in range(len(masked_logits)):
        #   try:
        #     pad_idx = ((masked_logits[i] == 0).nonzero(as_tuple=True)[0])[0].item()
        #   except:
        #     pad_idx = len(masked_logits[i])
        #   log_probs = 0
        #   word_list = []
        #   for j in range(0, pad_idx):
        #     cur_logit = masked_logits[i][j]
        #     distrib = Categorical(logits=cur_logit)
        #     next_word = distrib.sample()
        #     log_probs += distrib.log_prob(next_word)
        #     word_list.append(next_word)
        #   log_probs /= pad_idx
        #   log_probs_batch.append(log_probs)
        #   decoded_logits.append(tokenizer.decode(word_list, skip_special_tokens=True))
        # print(decoded_logits)
        # print(log_probs_batch)
        cat = Categorical(logits=masked_logits)
        samplebatch = cat.sample()
        log_probs_batch = cat.log_prob(samplebatch)
        log_probs_batch = torch.mean(log_probs_batch,1)
        decoded_logits = tokenizer.batch_decode(samplebatch, skip_special_tokens=True)
        bleu_scores = [metric.compute(predictions=[decoded_logits[i]], references=[decoded_labels[i]])["score"] for i in range(len(decoded_labels))]
        # self.bleu.append(sum(bleu_scores))
        # self.counter += len(bleu_scores)
        # avg_bleu = sum(self.bleu)/self.counter
        # bleu_scores = [score - avg_bleu for score in bleu_scores]
        
        # compute custom loss (suppose one has 3 labels with different weights)
        # probs = []
        # loss_fct = nn.CrossEntropyLoss()
        # for i in range(len(logits)):
        #   ce_loss = loss_fct(logits[i], labels[i])
        #   probs.append(ce_loss)
        # sys.exit(1)
        # loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        # print(loss)
        loss = sum([-log_prob*bleu_score for log_prob, bleu_score in zip(log_probs_batch, bleu_scores)])
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 909777
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 28431
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-2000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-2500/toke

RuntimeError: ignored

In [ ]:
print("Hello")

Hello


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/11785-Final/t5-20-bleu/checkpoint.pth'))
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
model = model.cuda()

In [ ]:
from tqdm.notebook import tqdm
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  output = model.generate(torch.LongTensor(data["input_ids"]).cuda().unsqueeze(0))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


14.615238166881701


In [ ]:
from tqdm.notebook import tqdm
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  output = model.generate(torch.LongTensor(data["input_ids"]).cuda().unsqueeze(0))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


14.615238166881701


In [ ]:
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 8.0 MB/s 


In [ ]:
from evaluate import load
bertscore = load("bertscore")
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  output = model.generate(torch.LongTensor(data["input_ids"]).cuda().unsqueeze(0))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  be = bertscore.compute(predictions=[tr], references=[label],lang='de')['f1'][0]
  scores+=be

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "l

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/pytorch_model.bin
All the weights of BertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


0.7768142389537003


In [ ]:
import evaluate
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from tqdm.notebook import tqdm
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  output = model.generate(torch.LongTensor(data["input_ids"]).cuda().unsqueeze(0))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  meteors = meteor.compute(predictions=[tr], references=[[label]])["meteor"]
  scores+=meteors

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


0.3363266290718009


In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/11785-Final/t5-20-bleu/checkpoint.pth')

# Without Prefix and Collator

In [ ]:
from tqdm.notebook import tqdm
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  input = tokenizer(data["translation"]['en'],return_tensors="pt").input_ids.to("cuda")
  model = model.cpu()
  input = input.cpu()
  output = model.generate(torch.LongTensor(input))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

4.561105102137273


In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/910 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small").cuda()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/d78aea13fa7ecd06c29e3e46195d6341255065d5/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_b

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch import nn
from torch.distributions import Categorical
import sys, torch

class CustomTrainer(Seq2SeqTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bleu = []
        self.distributions = []
        self.counter = 0 

    def compute_loss(self, model, inputs, return_outputs=False):
        log_probs = []
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        attn_mask = torch.where(labels != -100, inputs['attention_mask'], 0)
        masked_labels = labels*attn_mask
        # decoded_labels = []
        # for i in range(len(masked_labels)):
        #   decoded_labels.append([tokenizer.decode(masked_labels[i], skip_special_tokens=True)])
        decoded_labels = tokenizer.batch_decode(masked_labels,skip_special_tokens=True)
        decoded_labels = [[i] for i in decoded_labels]

        attn_mask = torch.unsqueeze(attn_mask, 2)
        masked_logits = logits*attn_mask
        decoded_logits = []
        log_probs_batch = []
        # for i in range(len(masked_logits)):
        #   try:
        #     pad_idx = ((masked_logits[i] == 0).nonzero(as_tuple=True)[0])[0].item()
        #   except:
        #     pad_idx = len(masked_logits[i])
        #   log_probs = 0
        #   word_list = []
        #   for j in range(0, pad_idx):
        #     cur_logit = masked_logits[i][j]
        #     distrib = Categorical(logits=cur_logit)
        #     next_word = distrib.sample()
        #     log_probs += distrib.log_prob(next_word)
        #     word_list.append(next_word)
        #   log_probs /= pad_idx
        #   log_probs_batch.append(log_probs)
        #   decoded_logits.append(tokenizer.decode(word_list, skip_special_tokens=True))
        # print(decoded_logits)
        # print(log_probs_batch)
        cat = Categorical(logits=masked_logits)
        samplebatch = cat.sample()
        log_probs_batch = cat.log_prob(samplebatch)
        log_probs_batch = torch.mean(log_probs_batch,1)
        decoded_logits = tokenizer.batch_decode(samplebatch, skip_special_tokens=True)
        bleu_scores = [metric.compute(predictions=[decoded_logits[i]], references=[decoded_labels[i]])["score"] for i in range(len(decoded_labels))]
        # self.bleu.append(sum(bleu_scores))
        # self.counter += len(bleu_scores)
        # avg_bleu = sum(self.bleu)/self.counter
        # bleu_scores = [score - avg_bleu for score in bleu_scores]
        
        # compute custom loss (suppose one has 3 labels with different weights)
        # probs = []
        # loss_fct = nn.CrossEntropyLoss()
        # for i in range(len(logits)):
        #   ce_loss = loss_fct(logits[i], labels[i])
        #   probs.append(ce_loss)
        # sys.exit(1)
        # loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        # print(loss)
        loss = sum([-log_prob*bleu_score for log_prob, bleu_score in zip(log_probs_batch, bleu_scores)])
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 909777
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 28431
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-27000] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-27500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/

Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-12500
Configuration saved in ./results/checkpoint-12500/config.json
Model weights saved in ./results/checkpoint-12500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-12500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-12500/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-11000] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-13000
Configuration saved in ./results/checkpoint-13000/config.json
Model weights saved in ./results/checkpoint-13000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-13000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-13000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-11500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-13500
Configuration saved in ./results/checkpoint-13500/config.json
Model weights saved in ./results

RuntimeError: ignored

In [ ]:
from tqdm.notebook import tqdm
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  input = tokenizer(data["translation"]['en'],return_tensors="pt").input_ids.to("cuda")
  model = model.cpu()
  input = input.cpu()
  output = model.generate(torch.LongTensor(input))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


14.087841188853858


# DON'T RUN

In [ ]:
model

In [ ]:
model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
tokenizer.to("cuda")

AttributeError: ignored

In [ ]:

# model_checkpoint = "results/checkpoint-284000"
translator = pipeline("translation_en_to_de", model=model,tokenizer=tokenizer)


input_ids = tokenizer("translate English to German: how are you", return_tensors="pt").input_ids.to('cuda')
# print(translator("How are you?"))
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

wie sind Sie


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 300 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
model.push_to_hub("t5-11785-bert-reinforce")

Configuration saved in /tmp/tmpi_fei6mw/config.json
Model weights saved in /tmp/tmpi_fei6mw/pytorch_model.bin
Uploading the following files to tgummadi/t5-11785-bert-reinforce: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/tgummadi/t5-11785-bert-reinforce/commit/89f132fbf43c2890e29b90bc23379d1dafb68a24', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='89f132fbf43c2890e29b90bc23379d1dafb68a24', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_datasets["validation"]

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 434
})

In [ ]:

# model_checkpoint = "results/checkpoint-284000"
translator = pipeline("translation_en_to_de", model=model,tokenizer=tokenizer)


input_ids = tokenizer("translate English to German: how are you", return_tensors="pt").input_ids.to('cuda')
# print(translator("How are you?"))
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
bleu_scores = [round(metric.compute(predictions=decoded_logits, references=decoded_labels)["score"], 1)]

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  output = model.generate(torch.LongTensor(data["input_ids"]).cuda().unsqueeze(0))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)

  

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 300 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


22.486928445611944


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("tgummadi/t5-11785",trust_remote_code=True,revision="221894f4bc75397b69f2f60d12ca9918488ae63b").cuda()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--tgummadi--t5-11785/snapshots/221894f4bc75397b69f2f60d12ca9918488ae63b/config.json
Model config T5Config {
  "_name_or_path": "tgummadi/t5-11785",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_

In [ ]:
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  input = tokenizer("translate English to German: "+data["translation"]['en'],return_tensors="pt").input_ids.to("cuda")
  model = model.cpu()
  input = input.cpu()
  output = model.generate(torch.LongTensor(input))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)


  0%|          | 0/434 [00:00<?, ?it/s]

15.16423070709688


In [ ]:
from transformers import pipeline
# model_checkpoint = "results/checkpoint-284000"
translator = pipeline("translation_en_to_de", model=model,tokenizer=tokenizer)


input_ids = tokenizer("how are you", return_tensors="pt").input_ids.to('cuda')
# print(translator("How are you?"))
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Wie bist du


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 300 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
len(tokenized_datasets["validation"])

434

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small",trust_remote_code=True).cuda()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/d78aea13fa7ecd06c29e3e46195d6341255065d5/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_b

In [ ]:
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  input = tokenizer("translate English to German: "+data["translation"]['en'],return_tensors="pt").input_ids.to("cuda")
  model = model.cpu()
  input = input.cpu()
  output = model.generate(torch.LongTensor(input))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bleu = metric.compute(predictions=[tr], references=[[label]])["score"]
  scores+=bleu

scores /= len(tokenized_datasets["validation"])
print(scores)

  

  0%|          | 0/434 [00:00<?, ?it/s]

15.163248346253972


In [ ]:

# model_checkpoint = "results/checkpoint-284000"
translator = pipeline("translation_en_to_de", model=model,tokenizer=tokenizer)

tokenizer.decode(data["validation"]['input_ids'])
input_ids = tokenizer("translate English to German: how are you", return_tensors="pt").input_ids.to('cuda')
# print(translator("How are you?"))
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
raw_datasets = load_dataset("wmt16", "de-en")

  0%|          | 0/3 [00:00<?, ?it/s]